# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2)

In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
# Your code here
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)

y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [6]:
# Your code here
from sklearn.metrics import mean_squared_error
residuals = np.sum((y_test - y_hat_test)**2)
print(f"Residuals: {residuals}")
print()

mse_test = mean_squared_error(y_test, y_hat_test)
mse_test2 = residuals/len(y_test)
print(f"Testing MSE  : {mse_test}")
print(f"Testing MSE 2: {mse_test2}")

Residuals: 56.54305211640686

Testing MSE  : 0.19364058943974952
Testing MSE 2: 0.19364058943974952


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [7]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    df = pd.DataFrame(data=data)
    output = []
    
    n = len(df)
    leftovers = n % k
    fold_size = n // k
    
    # initialize idx
    idx = 0
    for i in range(1, k+1):
        # when i is less than leftovers, increase fold_size by 1
        if i <= leftovers:
            output.append(df.loc[idx:(idx+fold_size), :])
            idx += fold_size+1
        # once all the leftovers are assigned into the first few folds
        # split the rest into fold_size sized subsets
        else:
            output.append(df.loc[idx:(idx+fold_size-1), :])
            idx += fold_size
        
    return output

### Apply it to the Ames Housing data

In [8]:
# Make sure to concatenate the data again
ames_data = pd.concat([X, y], axis=1)
ames_data.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
0,-0.133185,-0.803295,0.529078,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.559876
1,0.113403,0.418442,-0.381715,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.212692
2,0.419917,-0.576363,0.659449,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.733795
3,0.103311,-0.439137,0.541326,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.437232
4,0.878108,0.112229,1.281751,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.014303


In [9]:
# Apply kfolds() to ames_data with 5 folds
num_folds = 5
folds = kfolds(ames_data, num_folds)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [10]:
test_mses = []
train_mses = []
k=5

for n in range(k):
    # Split in train and test for the fold
#     remaining = list(set(range(k)) - set([n]))
#     train_lst = [folds[t] for t in remaining]
    train = pd.concat([folds[t] for t in range(k) if t != n], axis=0)
        
    test = folds[n]
    
    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']
    
    X_test = test.drop('SalePrice_log', axis=1)
    y_test = test['SalePrice_log']
    
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    
    # Predict for both train & test sets
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    
    # Evaluate Train and Test MSEs
    test_mses.append(mean_squared_error(y_test, y_hat_test))
    train_mses.append(mean_squared_error(y_train, y_hat_train))

print("Training MSEs:")
print(train_mses)
print()
print("Testing MSEs:")
print(test_mses)
print()
print(f"Average Testing MSE: {np.mean(test_mses)}")

Training MSEs:
[0.17170509651464658, 0.15507935685930538, 0.15659946326223304, 0.16134557666308721, 0.1516504855313168]

Testing MSEs:
[0.12431546148437427, 0.1935006463131313, 0.18910530431311187, 0.1707932525002693, 0.20742704588916944]

Average Testing MSE: 0.17702834210001125


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [11]:
# Your code here
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results

array([0.12431546, 0.19350065, 0.1891053 , 0.17079325, 0.20742705])

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [12]:
# Your code here
np.mean(cv_5_results)

0.17702834210001125

In [13]:
# This is the same as the average Testing MSE that we obtained above

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 